In [1]:
import re
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter

wn = WordNetLemmatizer()

resume = pd.read_csv("archive/Resume/Resume.csv")
#resume = resume.reindex(np.random.permutation(resume.index))
resume.head()


2023-05-30 20:04:10.344170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 20:04:10.960730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-30 20:04:11.732730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-30 20:04:11.751771: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [2]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
stopwords = nltk.corpus.stopwords.words('english')
def tokenize(input_doc):
    tokens = []
    wn = WordNetLemmatizer()
    cptoken = [wn.lemmatize(x).lower() for x in word_tokenize(input_doc) if x not in stopwords and \
                 not x.isnumeric() and x.isalpha() and len(x) >= 2]
        
    return cptoken

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

vectorizer = TfidfVectorizer(tokenizer=tokenize, analyzer="word")
bow_vectorizer = CountVectorizer(tokenizer=tokenize, analyzer="word")

corpus = resume.Resume_str
tfidf_matrix = vectorizer.fit_transform(corpus)
bow_matrix   = bow_vectorizer.fit_transform(corpus)
print(f"tfidf_matrix shape: {tfidf_matrix.shape}")
print(f"bow_matrix shape: {bow_matrix.shape}")

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
bow_cosine_sim = cosine_similarity(bow_matrix, bow_matrix)t
print(cosine_sim)
print(f"cosine_sim shape: {cosine_sim.shape}")

cosine_sim_lk = linear_kernel(tfidf_matrix, tfidf_matrix)

/home/arielg/anaconda3/envs/aistack/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


tfidf_matrix shape: (2484, 32351)
bow_matrix shape: (2484, 32351)
[[1.         0.25575024 0.23616097 ... 0.09187018 0.10855762 0.09396372]
 [0.25575024 1.         0.20146724 ... 0.07162671 0.1392705  0.0711357 ]
 [0.23616097 0.20146724 1.         ... 0.0621186  0.09232892 0.08819255]
 ...
 [0.09187018 0.07162671 0.0621186  ... 1.         0.0711868  0.17859443]
 [0.10855762 0.1392705  0.09232892 ... 0.0711868  1.         0.06674726]
 [0.09396372 0.0711357  0.08819255 ... 0.17859443 0.06674726 1.        ]]
cosine_sim shape: (2484, 2484)


## Utilizing TFIDF MAtrix for Semantic Search

In [4]:
len(list(vectorizer.get_feature_names_out()))

32351

In [5]:
from tqdm import tqdm

def inverted_index(words):
    """
        An ivnerted index of words (given word, find docID and idx)
    """
    inverted = {}
    for idx, word in enumerate(words):
        loc = inverted.setdefault(word, [])
        loc.append(idx)
    return inverted

def inverted_index_add(inverted, docID, doc_idx):
    for word in doc_idx.keys():
        loc = doc_idx[word]
        indices = inverted.setdefault(word, {})
        indices[docID] = loc
        
    return inverted

corpus = resume.Resume_str
inverted_doc_idx = {}
word_corpus = {}
with tqdm(total=len(corpus)) as pbar:
    for docid, x in enumerate(corpus):
        words = tokenize(x)
        word_corpus[docid] = words
        inv_idx = inverted_index(words)
        inverted_index_add(inverted_doc_idx, docid, inv_idx)
        pbar.update(1)
        
data_frequency = {}
for word in inverted_doc_idx.keys():
    # number of docs that the word appear in
    data_frequency[word] = len(list(inverted_doc_idx[word].keys()))
    
total_vocab_size = len(data_frequency)

print(f"total words: {total_vocab_size}")

100%|████████████████████████████████████████████████████████████████████████████| 2484/2484 [00:13<00:00, 181.55it/s]

total words: 34893


In [6]:
res = [(k,v) for k,v in inverted_doc_idx['admin'].items()]
adminidx = list(vectorizer.get_feature_names_out()).index('admin')
tfidf_matrix[res[0][0], adminidx]

0.03696502883983455

In [14]:
# sample search by hand and check
# Get the feature name "admin" from the bow vector, look it up in the TF-IDF matrix, and pick up the first resume and check if it is relevant
adminidx = list(vectorizer.get_feature_names_out()).index('admin')
val = tfidf_matrix[:,adminidx].nonzero()
print(val)
tfidf_matrix[val[0], [adminidx]*len(val[0])]
resume.Resume_str.iloc[273, ][:200]


(array([  60,   61,   87,  223,  224,  232,  259,  267,  289,  298,  316,
        330,  521,  686,  702,  703,  741,  789,  935,  979,  991, 1115,
       1147, 1153, 1195, 1232, 1286, 1378, 1420, 1537, 1574, 1577, 1579,
       1584, 1608, 1656, 1687, 1768, 1815, 1843, 1861, 1863, 1916, 1999,
       2159, 2242, 2325, 2395], dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int32))


'         DIRECTOR OF INFORMATION TECHNOLOGY       Executive Profile     Innovative executive and technology professional with strong work ethic and excellent communication skills, experienced in high-'

In [8]:
# Make sure you run the above to get your tfidf mat first as we refer it internally
def ranked_search(query, firstx=10):
    tokens = tokenize(query)
    query_weights = {}
    # get all the weights of the documents in which the term existed
    #get documents that matches the key
    for mapword, wmap in inverted_doc_idx.items():
        appear_in_docs = list(wmap.keys())

        if mapword in tokens:
            # print(f"looking via: {tokens}")
            # print(f"found {mapword} in tokens for docs {appear_in_docs}")
            for docid in appear_in_docs:
                wordidx = list(vectorizer.get_feature_names_out()).index(mapword)
                tfidfval = tfidf_matrix[docid,wordidx]
                # we add the value onto that doc id. The more words scored for that doc, the heavier the weight
                query_weights[docid] = query_weights.get(docid,0) + tfidfval
                
    query_weights = sorted(query_weights.items(), key=lambda x:x[1], reverse=True)[:firstx]
    result = []
    for (docid, tfidfval) in query_weights:
        data = {
                    'Relevance': round(tfidfval*100,2),
                    'ID': docid, 
                    'Resume_str': resume.Resume_str.iloc[docid], 
                    'Category': resume.Category[docid]
                }
        result.append(data)
    result = pd.DataFrame(result)
    return result

In [18]:
%time ranked_search("HR")

CPU times: user 2.05 s, sys: 4.09 ms, total: 2.05 s
Wall time: 2.05 s


,Relevance,ID,Resume_str,Category
0,58.90,4,HR MANAGER Skill Highlights ...,HR
1,51.87,101,REGIONAL HR BUSINESS PARTNER Hu...,HR
2,51.10,58,HR CONSULTANT Summary C...,HR
3,49.76,92,GLOBAL HR MANAGER Summary ...,HR
4,46.55,85,SENIOR HR BUSINESS PARTNER ...,HR
5,46.51,31,HR GENERALIST Professional Prof...,HR
6,46.18,68,HR DIRECTOR Summary HR Prof...,HR
7,45.31,69,HR PROFESSIONAL Summary Dep...,HR
8,42.35,88,REGIONAL HR DEPUTY MANAGER Summ...,HR
9,42.09,65,HR CONSULTING Summary 7+ yea...,HR


## Cosine similarity matrix of a corpus
Cosine score is 0 (no similarity) and 1 (exact same)


${sim(A,B)}$ = ${\cos(\theta)}$ = ${ {A\cdot B} \over ||A||||B|| }$

## Cosine Similarity Check
At this point, you can use the cosine similarity to check what are the documents that are most similar to based on the highest values in each row. To get a better value, you may use LSI (Latent Semantic Indexing) instead.

In [9]:
bow_vectorizer.vocabulary_['director']

8224

In [10]:
# with this, we can find documents that are closest to it and group them together

# sorted_cosine_sim = pd.DataFrame(cosine_sim).apply(lambda x: np.argsort(x), axis=1, raw=True) 
# sorted_cosine_sim
pd_cosine_sim = pd.DataFrame(cosine_sim)

test_item = pd_cosine_sim.iloc[0]
closest = np.argsort(test_item)[::-1]
test_item_closest_val = test_item[closest][:10]
print(test_item_closest_val)

resume.Category[[0] + closest].head()

0       1.000000
1803    0.348938
187     0.328642
1902    0.321385
375     0.315933
2116    0.313284
1576    0.312016
863     0.302114
2318    0.299672
1324    0.298496
Name: 0, dtype: float64


0                HR
1803    ENGINEERING
187        DESIGNER
1902     ACCOUNTANT
375         TEACHER
Name: Category, dtype: object

To compare a new file, we simply generate its BoW word vector and recompute the similarity matrix